In [1]:
from llama_index.llms.ollama import Ollama
import re
import json
import pandas as pd
import sys
import os
import numpy as np
import spacy
from heapq import nlargest
from llama_index.core import PromptTemplate
sys.path.append(os.path.abspath("MHQA-ontology"))
from prompts.prompt_templates import *
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Tuple, Union
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from utils.RAG_process import rag_generate
from utils.RAG_process import refine_supporting_facts
import ast
from utils.iterative_reasoner import process_question_with_kg_awareness
from utils.KG_builder import build_rdflib_knowledge_graph,process_contexts_in_batches
import traceback
from datetime import datetime

In [ ]:

llm = Ollama(model="llama3.3", request_timeout= 600.0,keep_alive=-1)
Hotpot_train = json.load(open('dataset/hotpot/hotpot_train_v1.1.json'))

dataset_df = pd.DataFrame(Hotpot_train[:400])

dataset = Hotpot_train[:15]
dataset_df = pd.DataFrame(dataset)

In [15]:
RAG_context = rag_generate(dataset,llm)
dataset_df = pd.DataFrame(dataset)
refine_dataset = dataset_df[dataset_df['question'].isin(RAG_context['Question'])].reset_index(drop=True)
refined_df = refine_supporting_facts(RAG_context, RAG_context, refine_dataset)

def compare_facts(refined_df):
    """
    Compares if all Supporting Facts are in Retrieval Facts.
    Sets coverage_all to 1 if all are present, else 0.
    """
    df = refined_df.copy()
    
    # Function to check if all supporting facts are in retrieval facts
    def check_coverage(row):
        supporting = row['Supporting Facts']
        retrieval = row['Retrieval Facts']
        
        # Convert strings to lists if needed
        if isinstance(supporting, str):
            supporting = ast.literal_eval(supporting)
        if isinstance(retrieval, str):
            retrieval = ast.literal_eval(retrieval)
        
        # Convert lists to strings for simple comparison
        supporting_str = [str(item) for item in supporting]
        retrieval_str = [str(item) for item in retrieval]
        
        # Check if all supporting facts are in retrieval facts
        all_present = all(item in retrieval_str for item in supporting_str)
        return 1 if all_present else 0
    
    # Apply the function to each row
    df['coverage_all'] = df.apply(check_coverage, axis=1)
    
    return df

refined_df2 = compare_facts(refined_df)



INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
/mnt/scratch/users/ts1201/MHQA-ontology/utils/RAG_process.py:521: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coverage = pd.concat([coverage, new_row], ignore_index=True)
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat

In [85]:
dataset = Hotpot_train[:15]
dataset_df = pd.DataFrame(dataset)
RAG_context_1 = refined_df2[refined_df2['coverage_all'] == 1].reset_index(drop=True)
 
dataset_df = dataset_df.loc[dataset_df['supporting_facts'].isin(RAG_context_1['Supporting Facts'].to_numpy())]

In [86]:
import pandas as pd

def extract_supporting_fact_content(supporting_facts, dataset_df):
    """
    Extract text content for supporting facts from dataset_df.
    
    Args:
        supporting_facts: List of supporting facts in format [[title, sentence_idx], ...]
        dataset_df: DataFrame containing 'context' column with the 3D list structure
    
    Returns:
        List of text content for each supporting fact
    """
    results = []
    
    # Get the context data from dataset_df
    # Since we need the corresponding row, we need to find which row has the matching context
    for fact in supporting_facts:
        title = fact[0]
        sentence_idx = fact[1]
        content_found = False
        
        # Search through each row in dataset_df to find matching title
        for _, row in dataset_df.iterrows():
            context_data = row['context']
            
            # Search through the context data for the title
            for item in context_data:
                if item[0] == title:  # Found matching title
                    sentences = item[1]
                    if 0 <= sentence_idx < len(sentences):
                        results.append(sentences[sentence_idx])
                    else:
                        results.append(f"[Index {sentence_idx} out of range for '{title}']")
                    content_found = True
                    break
            
            if content_found:
                break
        
        if not content_found:
            results.append(f"[No context found for '{title}']")
    
    return results

# Function to apply to each row in RAG_context_1
def process_row(row, dataset_df):
    supporting_facts = row['Supporting Facts']
    return extract_supporting_fact_content(supporting_facts, dataset_df)

# Create the new column
RAG_context_1['Supporting fact result'] = RAG_context_1.apply(
    lambda row: process_row(row, dataset_df), axis=1
)

# Example usage:


In [89]:
RAG_context_1

,Question,gold_answer,Supporting Facts,SPARQL query,Retrieval Facts,Retrieval Result,coverage_all,coverage_rate,Supporting fact result
0,Which magazine was started first Arthur's Maga...,Arthur's Magazine,"[[Arthur's Magazine, 0], [First for Women, 0]]",SELECT ?magazine WHERE { Arthur's_Magazine has...,"[[First for Women, 0], [Radio City (Indian rad...","[{'title': 'First for Women', 'paragraphs': ['...",1,0.152174,[Arthur's Magazine (1844–1846) was an American...
1,The Oberoi family is part of a hotel company t...,Delhi,"[[Oberoi family, 0], [The Oberoi Group, 0]]",SELECT ?city WHERE { The_Oberoi_Family part_of...,"[[Hotel Tallcorn, 4], [Hotel Bond, 5], [Glennw...","[{'title': 'Oberoi family', 'paragraphs': ['Th...",1,0.282051,[The Oberoi family is an Indian family that is...
2,What nationality was James Henry Miller's wife?,American,"[[Peggy Seeger, 0], [Peggy Seeger, 1], [Ewan M...",SELECT ?nationality WHERE { James_Henry_Miller...,"[[Incest: From a Journal of Love, 1], [Peggy S...","[{'title': 'Jim Miller (Australian footballer,...",1,0.478261,"[Margaret ""Peggy"" Seeger (born June 17, 1935) ..."
3,Who was once considered the best kick boxer in...,Badr Hari,"[[Global Fighting Championship, 1], [Global Fi...",SELECT ?kickboxer WHERE { ?kickboxer has_profe...,"[[Badr Hari, 1], [Prosecution of gender-target...","[{'title': 'Badr Hari', 'paragraphs': [' Hari ...",1,0.375000,[ Fighters from around world on the roster inc...
4,"The Dutch-Belgian television series that ""Hous...",2006,"[[House of Anubis, 0], [Het Huis Anubis, 1]]",SELECT ?firstAirYear WHERE { House_of_Anubis i...,"[[Batibot, 7], [Batibot, 6], [Het Huis Anubis,...","[{'title': 'House of Anubis', 'paragraphs': ['...",1,0.377358,[House of Anubis is a mystery television serie...
5,"Fast Cars, Danger, Fire and Knives includes gu...",Jaime Meline,"[[Fast Cars, Danger, Fire and Knives, 2], [El-...",SELECT ?executive WHERE { Fast_Cars_Danger_Fir...,"[[El-P, 0], [Changes (Alyson Avenue album), 2]...","[{'title': 'Fast Cars, Danger, Fire and Knives...",1,0.285714,"[ Vocals are handled by Aesop Rock, with guest..."


In [ ]:


# Initialize an empty list to store results
result_set = []

# Process each item in RAG_context_1
for i in range(len(RAG_context_1)):
    start_time = datetime.now()
    question = RAG_context_1['Question'][i]
    
    try:
        all_facts = RAG_context_1['Retrieval Result'][i]
        context_data = RAG_context_1['Retrieval Result'][i]
        
        # Process knowledge graph
        knowledge_graph = process_contexts_in_batches(
            context_items=all_facts,
            llm=llm,
            prompt_template=prompt_template_context,
            batch_size=2  # Process 2 contexts at a time
        )
        
        kg = build_rdflib_knowledge_graph(knowledge_graph)
        kg.serialize('kg'+str(i,format='ttl')
        
    except Exception as e:
        # If error occurs, create a result dict with None values except for question
        error_result = {
            "question": question,
            "answer": None,
            "queries_tried": None,
            "kg_exploration": None,
            "evidence_source": None,
            "confidence": None,
            "full_response_for_final": None
        }
        result_set.append(error_result)
        
        # Print error information for debugging
        print(f"Error processing question {i}: {str(e)}")
        print(traceback.format_exc())


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://loc